# FastAPI - Python Types Introduction

**Source:** [FastAPI Official Docs - Python Types](https://fastapi.tiangolo.com/python-types/)

This notebook covers Python type hints - the foundation of FastAPI's functionality.

---

## Why Type Hints?

Type hints provide:
- **Editor support** (autocomplete, better IntelliSense)
- **Type checking** (catch errors before runtime)
- **Better documentation** (self-documenting code)

In FastAPI, type hints enable:
- Automatic data validation
- Data conversion
- API documentation (OpenAPI)
- Error handling

## 1. Simple Types

Basic Python types: `str`, `int`, `float`, `bool`, `bytes`

In [ ]:
# Example: Function with type hints
def get_full_name(first_name: str, last_name: str) -> str:
    full_name = first_name.title() + " " + last_name.title()
    return full_name

print(get_full_name("john", "doe"))

In [ ]:
# Multiple simple types
def get_items(item_a: str, item_b: int, item_c: float, item_d: bool, item_e: bytes):
    return item_a, item_b, item_c, item_d, item_e

result = get_items("text", 42, 3.14, True, b"bytes")
print(result)

In [ ]:
# Type hints help catch errors
def get_name_with_age(name: str, age: int) -> str:
    name_with_age = name + " is this old: " + str(age)  # Must convert age to str
    return name_with_age

print(get_name_with_age("Alice", 30))

## 2. Generic Types with Type Parameters

**Generic types** contain other types: `list`, `tuple`, `set`, `dict`

### Python 3.9+ Syntax
Use built-in types directly with square brackets: `list[str]`, `dict[str, int]`

### Python 3.6-3.8
Import from `typing`: `List[str]`, `Dict[str, int]`

### List

In [ ]:
# Python 3.9+: list[type]
def process_items(items: list[str]):
    for item in items:
        print(item.upper())  # Editor knows item is str

process_items(["apple", "banana", "cherry"])

### Tuple and Set

In [ ]:
# Tuple: fixed number of elements with specific types
# Set: collection of unique elements
def process_items(items_t: tuple[int, int, str], items_s: set[bytes]):
    return items_t, items_s

result = process_items((1, 2, "three"), {b"a", b"b", b"c"})
print(f"Tuple: {result[0]}")
print(f"Set: {result[1]}")

### Dict

In [ ]:
# Dict: dict[key_type, value_type]
def process_items(prices: dict[str, float]):
    for item_name, item_price in prices.items():
        print(f"{item_name}: ${item_price}")

process_items({"apple": 1.50, "banana": 0.75, "cherry": 2.00})

## 3. Union - Multiple Possible Types

**Python 3.10+:** Use `|` operator  
**Python 3.6-3.9:** Use `Union[type1, type2]` from `typing`

In [ ]:
# Python 3.10+ syntax
def process_item(item: int | str):
    print(f"Processing: {item} (type: {type(item).__name__})")

process_item(42)
process_item("hello")

In [ ]:
# Python 3.9+ alternative (for compatibility)
from typing import Union

def process_item_old(item: Union[int, str]):
    print(f"Processing: {item} (type: {type(item).__name__})")

process_item_old(100)
process_item_old("world")

## 4. Optional / Possibly None

When a value can be of a specific type OR `None`:

**Python 3.10+:** `str | None`  
**Python 3.9+:** `Optional[str]` or `Union[str, None]`

**Important:** `Optional` doesn't mean "optional parameter" - it means "can be None"

In [ ]:
# Python 3.10+ syntax
def say_hi(name: str | None = None):
    if name is not None:
        print(f"Hey {name}!")
    else:
        print("Hello World")

say_hi("Alice")
say_hi()
say_hi(name=None)

In [ ]:
# Python 3.9+ with Optional
from typing import Optional

def say_hi_optional(name: Optional[str] = None):
    if name is not None:
        print(f"Hey {name}!")
    else:
        print("Hello World")

say_hi_optional("Bob")
say_hi_optional()

In [ ]:
# Recommended: Union[str, None] (more explicit than Optional)
from typing import Union

def say_hi_union(name: Union[str, None] = None):
    if name is not None:
        print(f"Hey {name}!")
    else:
        print("Hello World")

say_hi_union("Charlie")
say_hi_union()

### Important Note on Optional

```python
# This parameter is REQUIRED but accepts None
def say_hi(name: str | None):  # No default value!
    print(f"Hey {name}!")

# say_hi()  # ❌ Error: missing required argument
say_hi(None)  # ✅ Works: None is valid
```
> given specific use-case, you can set default value for function argument implementation to avoid such an error
```python
def say_hi(name: str | None = None):  # No default value!
    print(f"Hey {name}!")

say_hi()  # ✅ Valid
```

## 5. Classes as Types

In [ ]:
class Person:
    def __init__(self, name: str):
        self.name = name

def get_person_name(one_person: Person) -> str:
    return one_person.name

# Create instance and use
alice = Person("Alice")
print(get_person_name(alice))

## 6. Pydantic Models

**Pydantic** is the data validation library FastAPI is built on.

Features:
- Data validation
- Automatic type conversion
- Clear error messages
- Editor support

In [ ]:
from datetime import datetime
from pydantic import BaseModel

class User(BaseModel):
    id: int
    name: str = "John Doe"  # Default value
    signup_ts: datetime | None = None
    friends: list[int] = []

# Data from external source (e.g., API request)
external_data = {
    "id": "123",  # String, but will be converted to int
    "signup_ts": "2017-06-01 12:22",  # String, will be converted to datetime
    "friends": [1, "2", b"3"],  # Mixed types, all converted to int
}

user = User(**external_data)
print(user)
print(f"User ID: {user.id} (type: {type(user.id).__name__})")
print(f"Signup: {user.signup_ts} (type: {type(user.signup_ts).__name__})")
print(f"Friends: {user.friends}")

In [ ]:
# Pydantic validates data
try:
    invalid_user = User(id="not_a_number")  # This will fail validation
except Exception as e:
    print(f"Validation error: {e}")

## 7. Type Hints with Metadata Annotations

**Python 3.9+:** `Annotated` allows adding metadata to type hints.

- First parameter: the **actual type**
- Rest: **metadata** for tools like FastAPI

Python itself ignores the metadata, but FastAPI uses it for validation, documentation, etc.

In [ ]:
from typing import Annotated

def say_hello(name: Annotated[str, "this is just metadata"]) -> str:
    return f"Hello {name}"

print(say_hello("World"))

# The type is still 'str', metadata is for tools

## Summary: What FastAPI Does with Type Hints

When you declare parameters with type hints in FastAPI:

### You Get:
1. **Editor support** - autocomplete and type checking
2. **Type checks** - catch errors early

### FastAPI Uses Them To:
1. **Define requirements** - path params, query params, headers, body, etc.
2. **Convert data** - from request to required type
3. **Validate data** - automatic error responses for invalid data
4. **Document API** - automatic OpenAPI/Swagger docs

All with **standard Python types** - no special decorators or classes needed!

## Quick Reference: Generic Types

Types that can contain other types:

| Python 3.9+ | Python 3.6-3.8 (typing) | Description |
|-------------|-------------------------|-------------|
| `list[str]` | `List[str]` | List of strings |
| `tuple[int, str]` | `Tuple[int, str]` | Tuple with int and str |
| `set[int]` | `Set[int]` | Set of integers |
| `dict[str, float]` | `Dict[str, float]` | Dict with str keys and float values |
| `int \| str` | `Union[int, str]` | Either int or str |
| `str \| None` | `Optional[str]` or `Union[str, None]` | String or None |

**Note:** For Python 3.10+, use `|` for unions. For earlier versions, use `Union` from `typing`.

## Practice Exercise

Try creating your own functions with type hints!

In [ ]:
# Exercise: Create a function that takes a list of numbers and returns their average
# Add proper type hints!

def calculate_average(numbers: list[float]) -> float:
    if not numbers:
        return 0.0
    return sum(numbers) / len(numbers)

# Test it
print(calculate_average([1.5, 2.5, 3.5, 4.5]))
print(calculate_average([]))

In [ ]:
# Exercise: Create a Pydantic model for a Product
from pydantic import BaseModel

class Product(BaseModel):
    id: int
    name: str
    price: float
    description: str | None = None
    in_stock: bool = True

# Test it
product_data = {
    "id": "1",
    "name": "Laptop",
    "price": "999.99",
}

product = Product(**product_data)
print(product)
print(f"Price type: {type(product.price).__name__}")

---

## Next Steps

Now that you understand Python type hints, you're ready to learn:
- [Concurrency and async/await](https://fastapi.tiangolo.com/async/)
- [FastAPI Tutorial - First Steps](https://fastapi.tiangolo.com/tutorial/first-steps/)

**Additional Resources:**
- [Pydantic Documentation](https://docs.pydantic.dev/)
- [mypy Cheat Sheet](https://mypy.readthedocs.io/en/latest/cheat_sheet_py3.html)
- [Python typing Module](https://docs.python.org/3/library/typing.html)